In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Ishita Lyall
101903127
COE15
Convo AI Assignment-1

In [ ]:
import cuml
import cupy as cp
import cudf
import numpy as np

In [ ]:
df_train = cudf.read_csv("../input/house-prices-advanced-regression-techniques/train.csv")
df_train

In [ ]:
df_test  = cudf.read_csv("../input/house-prices-advanced-regression-techniques/test.csv")
df_test

In [ ]:
df_train.info()

In [ ]:
df_train.isnull().sum()

Handling missing values

In [ ]:
missing_from_df_train = df_train.isnull().sum()/len(df_train)
missing_from_df_train
(missing_from_df_train>0.3).sum()

drop_from_train = missing_from_df_train[missing_from_df_train>0.3]
df_train.drop(columns = drop_from_train.keys().to_array(),axis = 1, inplace=True)

df_train.info()
df_train.reset_index(drop=True,inplace = True)

Encoding data

In [ ]:
categorical_features=[]
for cf in df_train.columns:
    if(df_train[cf].dtype=='O'):
        categorical_features.append(cf)
        
        
print("categorical_features : ",len(categorical_features))

Dealing with numerical data

In [ ]:
numerical_features=[]
for nf in df_train.columns:
    if(df_train[nf].dtype!='O'):
        numerical_features.append(nf)
        
        
print("numerical_features : ",len(numerical_features))

year_features = [feature for feature in numerical_features if 'Year' in feature or 'Yr' in feature ]
year_features

In [ ]:
discrete_features = [dft for dft in numerical_features if len(df_train[dft].unique()) < 25 and
                     dft not in year_features]
len(discrete_features)

In [ ]:
continuos_features  = [cft for cft in numerical_features if cft not in discrete_features and cft not in year_features]
len(continuos_features)

In [ ]:
df_train.info()

The number of unique rows (for encoding)

In [ ]:
for feature in categorical_features:
    print(f'The feature is {feature} and no of unique categories are {len(df_train[feature].unique())}')

Encoding the missing cateorical values and replace the ones in numerical category:

In [ ]:
categorical_features_null_values= [feature for feature in df_train.columns if df_train[feature].isnull().sum() > 0 
                                    and df_train[feature].dtype == 'O']

for feature in categorical_features_null_values :
    print(f"Amount of missing values in {feature} is {df_train[feature].isnull().sum()/len(df_train[feature])}")

In [ ]:
def replace_missing_categorical_features(dataset,features):
    data = dataset.copy()
    data[features] = data[features].fillna('Missing_values')
    return data

In [ ]:
df_train = replace_missing_categorical_features(df_train, categorical_features)

Replacing missing values with mean values:

In [ ]:
numerical_features_missing_values = [feature for feature in numerical_features if df_train[feature].isnull().sum() > 0]
numerical_features_missing_values

for feature in numerical_features_missing_values:
    df_train[feature] = df_train[feature].fillna(df_train[feature].mode())

Scaling data:

In [ ]:
from cuml.preprocessing import StandardScaler
s = StandardScaler()
df_train[numerical_features]=s.fit_transform(df_train[numerical_features])

df_train[numerical_features]

In [ ]:
from cuml.preprocessing import LabelEncoder
enc = LabelEncoder()
for feature in categorical_features:
    df_train[feature] = enc.fit_transform(df_train[feature])
   
df_train[categorical_features].head()

In [ ]:
for feature in ['YearBuilt','YearRemodAdd','GarageYrBlt']:
    df_train[feature] = df_train['YrSold'] - df_train[feature]

In [ ]:
df_train.info()

Feature selection:

In [ ]:
df_train = df_train[["Street","OpenPorchSF","WoodDeckSF","OverallQual","YearBuilt","YearRemodAdd","ExterQual","TotalBsmtSF",
                     "1stFlrSF","GrLivArea","PoolArea","Fireplaces","RoofStyle",
                     "FullBath","TotRmsAbvGrd","GarageCars","GarageArea",
                   "MSZoning", "Utilities","BldgType","Heating","KitchenQual","SaleCondition","LandSlope","SalePrice"]]

In [ ]:
from sklearn.model_selection import train_test_split
X = df_train.drop('SalePrice',axis=1)
Y = df_train['SalePrice']
x_train, x_valid, y_train, y_valid = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
y_valid.isnull().sum()

Comparison:

In [ ]:
from cuml.linear_model import LinearRegression

algo = ['svd', 'eig', 'qr', 'svd-qr', 'svd-jacobi']

for i in algo: 
    lr = LinearRegression(fit_intercept = True, normalize = False,
                      algorithm = i)

    reg = lr.fit(x_train,y_train)

    print(reg.coef_)

    preds=lr.predict(x_valid)
    
    print(preds)
    
    MSE=cuml.metrics.regression.mean_squared_error(y_valid,preds)
    MAE=cuml.metrics.regression.mean_absolute_error(y_valid,preds)
    R2_Score=cuml.metrics.regression.r2_score(y_valid,preds)
    

    print(f"for algo {i} MSE is {MSE}  MAE is {MAE} r2_score is {R2_Score}")